In [ ]:
import requests
import pandas as pd
from io import StringIO
import folium

In [108]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [109]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)

csv_file = StringIO(resp.text)
spacex_df = pd.read_csv(csv_file)

In [110]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [111]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [112]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [113]:
data = {'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
        'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
        'Long': [-80.577356, -80.576820, -80.646895, -120.610745]}

launch_sites_df = pd.DataFrame(data)

center_lat = launch_sites_df['Lat'].mean()
center_long = launch_sites_df['Long'].mean()

site_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

##### Task 1: Mark all launch sites on a map

In [114]:
data = {'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
        'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
        'Long': [-80.577356, -80.576820, -80.646895, -120.610745]}

launch_sites_df = pd.DataFrame(data)


center_lat = launch_sites_df['Lat'].mean()
center_long = launch_sites_df['Long'].mean()


site_map = folium.Map(location=[center_lat, center_long], zoom_start=5)


for index, row in launch_sites_df.iterrows():
    
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',
        fill=True).add_child(folium.Popup(row['Launch Site']))

    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(row['Launch Site'])))
    
    
    site_map.add_child(circle)
    site_map.add_child(marker)

In [115]:
site_map

##### Task 2: Mark the success/failed launches for each site on the map

In [116]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [117]:
marker_cluster = MarkerCluster()

In [118]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

for index, record in spacex_df.iterrows():
    
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

##### # TASK 3: Calculate the distances between a launch site to its proximities

In [119]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [120]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [121]:
coastline_coords = [
    [28.56229, -80.56774],
    [28.56321, -80.56793], 
    [28.5723, -80.60649],
    [34.63225, -120.62638]
]

In [122]:

for index, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    site_lat = row['Lat']
    site_lon = row['Long']
    
    
    min_distance = float('inf')
    closest_coastline = None

    for coast_coords in coastline_coords:
        distance = calculate_distance(site_lat, site_lon, coast_coords[0], coast_coords[1])
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast_coords
    
    
    folium.Marker(
        location=[site_lat, site_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(site_name)
        )
    ).add_to(site_map)

    
    if closest_coastline:       
        folium.PolyLine(
            locations=[[site_lat, site_lon], closest_coastline],
            weight=2,
            color='blue'
        ).add_to(site_map)
        
        
        folium.Marker(
            location=closest_coastline,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(min_distance)
            )
        ).add_to(site_map)

In [123]:
site_map

##### Closest Railroad, Highway, City

In [124]:

railway_coords = [
    [28.57227, -80.58526],
    [28.57227, -80.58526], 
    [28.57227, -80.58526],
    [34.63267, -120.62502]
]


highway_coords = [
    [28.57191, -80.57281],
    [28.57191, -80.57281], 
    [28.57191, -80.57281],
    [34.6827, -120.60379]
]


city_coords = [
    [28.07772, -80.60788],
    [28.07772, -80.60788], 
    [28.07772, -80.60788],
    [34.95271, -120.43598]
]


def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0 

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


for index, row in launch_sites_df.iterrows():
    
    site_name = row['Launch Site']
    site_lat = row['Lat']
    site_lon = row['Long']


    min_distance_railway = float('inf')
    closest_railway = None

    for railway in railway_coords:
        distance = calculate_distance(site_lat, site_lon, railway[0], railway[1])
        if distance < min_distance_railway:
            min_distance_railway = distance
            closest_railway = railway


    min_distance_highway = float('inf')
    closest_highway = None

    for highway in highway_coords:
        distance = calculate_distance(site_lat, site_lon, highway[0], highway[1])
        if distance < min_distance_highway:
            min_distance_highway = distance
            closest_highway = highway


    min_distance_city = float('inf')
    closest_city = None

    for city in city_coords:
        distance = calculate_distance(site_lat, site_lon, city[0], city[1])
        if distance < min_distance_city:
            min_distance_city = distance
            closest_city = city
    

    marker = folium.Marker(
        location=[site_lat, site_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(site_name)
        )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)


    if closest_railway:
        folium.PolyLine(
            locations=[[site_lat, site_lon], closest_railway],
            weight=2,
            color='hotpink'
        ).add_to(site_map)

        railway_marker = folium.Marker(
            location=closest_railway,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#0000FF;"><b>{:.2f} KM</b></div>'.format(min_distance_railway)
            )
        )
        railway_marker.add_to(site_map)


    if closest_highway:
        folium.PolyLine(
            locations=[[site_lat, site_lon], closest_highway],
            weight=2,
            color='green'
        ).add_to(site_map)

        highway_marker = folium.Marker(
            location=closest_highway,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#FF5733;"><b>{:.2f} KM</b></div>'.format(min_distance_highway)
            )
        )
        highway_marker.add_to(site_map)


    if closest_city:
        folium.PolyLine(
            locations=[[site_lat, site_lon], closest_city],
            weight=2,
            color='purple'
        ).add_to(site_map)

        city_marker = folium.Marker(
            location=closest_city,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#8E44AD;"><b>{:.2f} KM</b></div>'.format(min_distance_city)
            )
        )
        city_marker.add_to(site_map)

In [125]:
site_map